In [14]:
# ==========================================
# ⚙️ CELL 1: USER CONFIGURATION
# ==========================================

# 1. CHOOSE LEAGUE (Must match the table name in your DB)
# Examples: 'premier_league', 'serie_a', 'bundesliga', 'la_liga'
LEAGUE_TABLE = "premier_league"

# 2. CHOOSE MODE: "MATCH" or "LEAGUE"
ANALYSIS_MODE = "MATCH"

# 3. SELECT TEAMS (Only used if Mode is "MATCH")
HOME_TEAM = "Arsenal"
AWAY_TEAM = "Liverpool"

# --- SYSTEM SETUP (Do not edit below) ---
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')

# Handle Database Path
if Path("football_2526.db").exists():
    DB_PATH = Path("football_2526.db") # Colab
else:
    DB_PATH = Path("../data/football_2526.db") # Local PC

def check_db_tables():
    """Helper to show available leagues."""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]
    conn.close()
    return tables

# Validate Connection
if not DB_PATH.exists():
    print(f"❌ Error: Database not found at {DB_PATH}")
else:
    tables = check_db_tables()
    if LEAGUE_TABLE not in tables:
        print(f"⚠️ Warning: Table '{LEAGUE_TABLE}' not found.")
        print(f"ℹ️ Available Leagues: {tables}")
    else:
        print(f"✅ Setup Complete. Connected to: {LEAGUE_TABLE.upper()}")
        print(f"📊 Mode: {ANALYSIS_MODE}")

✅ Setup Complete. Connected to: PREMIER_LEAGUE
📊 Mode: MATCH


In [15]:
# ==========================================
# ⚙️ CELL 2: DATA ENGINE
# ==========================================

def get_raw_data(league_table):
    """Fetches the entire season data for the selected league."""
    conn = sqlite3.connect(DB_PATH)
    # Note: Table name cannot be parameterized in SQLite, so we use f-string safely here
    query = f"""
    SELECT Date, HomeTeam, AwayTeam,
           FTHG, FTAG,
           HS, "AS" as AS_shots, HST, AST,
           HC, AC,
           HF, AF,
           HY, AY
    FROM {league_table}
    """
    try:
        df = pd.read_sql(query, conn)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        return df
    except Exception as e:
        print(f"❌ Database Error: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

def generate_league_stats(df):
    """Aggregates stats for the League Overview mode."""
    # Home Form
    home = df.groupby('HomeTeam').agg({
        'FTHG': 'mean', 'FTAG': 'mean', 'HST': 'mean', 'HC': 'mean', 'HY': 'sum'
    }).reset_index().rename(columns={'HomeTeam': 'Team'})

    # Away Form
    away = df.groupby('AwayTeam').agg({
        'FTAG': 'mean', 'FTHG': 'mean', 'AST': 'mean', 'AC': 'mean', 'AY': 'sum'
    }).reset_index().rename(columns={'AwayTeam': 'Team'})

    # Combine
    stats = pd.merge(home, away, on='Team')

    # Metrics
    stats['Goals_Scored_Avg'] = (stats['FTHG_x'] + stats['FTAG_y']) / 2
    stats['Goals_Conceded_Avg'] = (stats['FTAG_x'] + stats['FTHG_y']) / 2
    stats['SOT_Avg'] = (stats['HST'] + stats['AST']) / 2
    stats['Corners_Avg'] = (stats['HC'] + stats['AC']) / 2
    stats['Total_Cards'] = stats['HY'] + stats['AY']

    return stats.round(2)

def get_defensive_shots(df, team_name):
    """Helper for Charts 3 & 4 (Exact logic from your file)."""
    data = []
    for index, row in df.iterrows():
        if row['HomeTeam'] == team_name: # Team is Home, Opponent is Away
            data.append({'Date': row['Date'], 'Location': 'Home', 'Opponent': row['AwayTeam'], 'Metric': 'Shots Allowed', 'Value': row['AS_shots']})
            data.append({'Date': row['Date'], 'Location': 'Home', 'Opponent': row['AwayTeam'], 'Metric': 'SOT Allowed', 'Value': row['AST']})
        if row['AwayTeam'] == team_name: # Team is Away, Opponent is Home
            data.append({'Date': row['Date'], 'Location': 'Away', 'Opponent': row['HomeTeam'], 'Metric': 'Shots Allowed', 'Value': row['HS']})
            data.append({'Date': row['Date'], 'Location': 'Away', 'Opponent': row['HomeTeam'], 'Metric': 'SOT Allowed', 'Value': row['HST']})
    return pd.DataFrame(data).sort_values(by='Date')

print("✅ Data Engine Ready.")

✅ Data Engine Ready.


In [16]:
# ==========================================
# 📈 CELL 3: VISUALIZATION GENERATOR
# ==========================================

# 1. Load Data
df = get_raw_data(LEAGUE_TABLE)

if df.empty:
    print("❌ No data loaded.")

elif ANALYSIS_MODE == "LEAGUE":
    print(f"🏆 GENERATING SEASON OVERVIEW: {LEAGUE_TABLE.upper()}")
    stats = generate_league_stats(df)

    # Chart 1: Attack vs Defense Quality (Scatter)
    fig1 = px.scatter(
        stats, x='Goals_Scored_Avg', y='Goals_Conceded_Avg',
        text='Team', size='SOT_Avg', color='SOT_Avg',
        title=f'<b>{LEAGUE_TABLE.upper()}: Attack vs Defense Quality</b><br><i>(Size = Shots on Target)</i>',
        labels={'Goals_Scored_Avg': 'Goals Scored / Game', 'Goals_Conceded_Avg': 'Goals Conceded / Game'},
        template='plotly_white', height=700
    )
    fig1.update_traces(textposition='top center')
    fig1.add_hline(y=stats['Goals_Conceded_Avg'].mean(), line_dash="dot", annotation_text="Avg Defense")
    fig1.add_vline(x=stats['Goals_Scored_Avg'].mean(), line_dash="dot", annotation_text="Avg Attack")
    fig1.show()

    # Chart 2: Corner Kick Kings
    fig2 = px.bar(
        stats.sort_values('Corners_Avg', ascending=False),
        x='Team', y='Corners_Avg', color='Corners_Avg',
        title=f'<b>Set Piece Threat: Average Corners per Game</b>',
        template='plotly_white', height=500
    )
    fig2.show()

    # Chart 3: Discipline (Naughty List)
    fig3 = px.bar(
        stats.sort_values('Total_Cards', ascending=False),
        x='Team', y='Total_Cards', color='Total_Cards', color_continuous_scale='Reds',
        title=f'<b>Discipline: Total Yellow Cards</b>',
        template='plotly_white', height=500
    )
    fig3.show()

elif ANALYSIS_MODE == "MATCH":
    print(f"🚀 GENERATING 7-CHART REPORT FOR: {HOME_TEAM} vs {AWAY_TEAM}\n")

    # ==============================================================================
    # CHART 1: ATTACK EFFICIENCY (HEAD-TO-HEAD TIMELINE)
    # ==============================================================================
    plot_data = []
    selected_teams = [HOME_TEAM, AWAY_TEAM]

    for index, row in df.iterrows():
        # Home Team Stats
        if row['HomeTeam'] in selected_teams:
            plot_data.append({'Date': row['Date'], 'Team': row['HomeTeam'], 'Opponent': row['AwayTeam'], 'Metric': 'Total Shots', 'Value': row['HS']})
            plot_data.append({'Date': row['Date'], 'Team': row['HomeTeam'], 'Opponent': row['AwayTeam'], 'Metric': 'Shots on Target', 'Value': row['HST']})
        # Away Team Stats
        if row['AwayTeam'] in selected_teams:
            plot_data.append({'Date': row['Date'], 'Team': row['AwayTeam'], 'Opponent': row['HomeTeam'], 'Metric': 'Total Shots', 'Value': row['AS_shots']})
            plot_data.append({'Date': row['Date'], 'Team': row['AwayTeam'], 'Opponent': row['HomeTeam'], 'Metric': 'Shots on Target', 'Value': row['AST']})

    df_plot = pd.DataFrame(plot_data).sort_values(by='Date')

    if not df_plot.empty:
        fig = px.line(
            df_plot, x='Date', y='Value', color='Team', line_dash='Metric', markers=True,
            title=f'<b>1. Attack Efficiency Comparison</b><br><i>(Solid = Total Shots | Dashed = On Target)</i>',
            labels={'Value': 'Count', 'Date': 'Date'}, hover_data=['Opponent'], template='plotly_white', height=500
        )
        fig.update_layout(hovermode='x unified')
        fig.show()

    # ==============================================================================
    # CHART 2: SHOTS FORM (HOME vs AWAY SPECIFIC)
    # ==============================================================================
    plot_data = []
    for index, row in df.iterrows():
        if row['HomeTeam'] == HOME_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{HOME_TEAM} (Home)", 'Opponent': row['AwayTeam'], 'Metric': 'Total Shots', 'Value': row['HS']})
            plot_data.append({'Date': row['Date'], 'Team': f"{HOME_TEAM} (Home)", 'Opponent': row['AwayTeam'], 'Metric': 'On Target', 'Value': row['HST']})
        if row['AwayTeam'] == AWAY_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{AWAY_TEAM} (Away)", 'Opponent': row['HomeTeam'], 'Metric': 'Total Shots', 'Value': row['AS_shots']})
            plot_data.append({'Date': row['Date'], 'Team': f"{AWAY_TEAM} (Away)", 'Opponent': row['HomeTeam'], 'Metric': 'On Target', 'Value': row['AST']})

    df_plot = pd.DataFrame(plot_data).sort_values(by='Date')
    if not df_plot.empty:
        fig = px.line(
            df_plot, x='Date', y='Value', color='Team', line_dash='Metric', markers=True,
            title=f'<b>2. Shot Form: {HOME_TEAM} (Home) vs. {AWAY_TEAM} (Away)</b>',
            labels={'Value': 'Shots', 'Date': 'Date'}, hover_data=['Opponent'], template='plotly_white', height=500
        )
        fig.update_layout(hovermode='x unified')
        fig.show()

    # ==============================================================================
    # CHART 3 & 4: DEFENSIVE RECORDS (SHOTS CONCEDED)
    # ==============================================================================
    # Home Team Defense
    df_home_def = get_defensive_shots(df, HOME_TEAM)
    if not df_home_def.empty:
        fig = px.line(df_home_def, x='Date', y='Value', color='Location', line_dash='Metric', markers=True,
                      title=f'<b>3. Defensive Record: {HOME_TEAM}</b> (Shots Allowed)', hover_data=['Opponent'], template='plotly_white', height=400)
        fig.update_layout(hovermode='x unified')
        fig.show()

    # Away Team Defense
    df_away_def = get_defensive_shots(df, AWAY_TEAM)
    if not df_away_def.empty:
        fig = px.line(df_away_def, x='Date', y='Value', color='Location', line_dash='Metric', markers=True,
                      title=f'<b>4. Defensive Record: {AWAY_TEAM}</b> (Shots Allowed)', hover_data=['Opponent'], template='plotly_white', height=400)
        fig.update_layout(hovermode='x unified')
        fig.show()

    # ==============================================================================
    # CHART 5: CORNERS BATTLE (HOME vs AWAY FORM)
    # ==============================================================================
    plot_data = []
    for index, row in df.iterrows():
        if row['HomeTeam'] == HOME_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{HOME_TEAM} (Home)", 'Opponent': row['AwayTeam'], 'Value': row['HC']})
        if row['AwayTeam'] == AWAY_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{AWAY_TEAM} (Away)", 'Opponent': row['HomeTeam'], 'Value': row['AC']})

    df_plot = pd.DataFrame(plot_data).sort_values(by='Date')
    if not df_plot.empty:
        fig = px.line(
            df_plot, x='Date', y='Value', color='Team', markers=True,
            title=f'<b>5. Corners Won: {HOME_TEAM} (Home) vs. {AWAY_TEAM} (Away)</b>',
            labels={'Value': 'Corners', 'Date': 'Date'}, hover_data=['Opponent'], template='plotly_white', height=500
        )
        fig.update_layout(hovermode='x unified')
        fig.show()

    # ==============================================================================
    # CHART 6: CORNERS CONCEDED (DEFENSIVE WEAKNESS)
    # ==============================================================================
    plot_data = []
    for index, row in df.iterrows():
        if row['HomeTeam'] == HOME_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{HOME_TEAM} (Home)", 'Opponent': row['AwayTeam'], 'Value': row['AC']}) # Opponent corners
        if row['AwayTeam'] == AWAY_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{AWAY_TEAM} (Away)", 'Opponent': row['HomeTeam'], 'Value': row['HC']}) # Opponent corners

    df_plot = pd.DataFrame(plot_data).sort_values(by='Date')
    if not df_plot.empty:
        fig = px.line(
            df_plot, x='Date', y='Value', color='Team', markers=True,
            title=f'<b>6. Corners Conceded: {HOME_TEAM} (Home) vs. {AWAY_TEAM} (Away)</b>',
            labels={'Value': 'Corners Allowed', 'Date': 'Date'}, hover_data=['Opponent'], template='plotly_white', height=500
        )
        fig.update_layout(hovermode='x unified')
        fig.show()

    # ==============================================================================
    # CHART 7: DISCIPLINE (FOULS & CARDS)
    # ==============================================================================
    plot_data = []
    for index, row in df.iterrows():
        if row['HomeTeam'] == HOME_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{HOME_TEAM} (Home)", 'Opponent': row['AwayTeam'], 'Metric': 'Fouls', 'Value': row['HF']})
            plot_data.append({'Date': row['Date'], 'Team': f"{HOME_TEAM} (Home)", 'Opponent': row['AwayTeam'], 'Metric': 'Yellow Cards', 'Value': row['HY']})
        if row['AwayTeam'] == AWAY_TEAM:
            plot_data.append({'Date': row['Date'], 'Team': f"{AWAY_TEAM} (Away)", 'Opponent': row['HomeTeam'], 'Metric': 'Fouls', 'Value': row['AF']})
            plot_data.append({'Date': row['Date'], 'Team': f"{AWAY_TEAM} (Away)", 'Opponent': row['HomeTeam'], 'Metric': 'Yellow Cards', 'Value': row['AY']})

    df_plot = pd.DataFrame(plot_data).sort_values(by='Date')
    if not df_plot.empty:
        fig = px.line(
            df_plot, x='Date', y='Value', color='Team', line_dash='Metric', markers=True,
            title=f'<b>7. Discipline: Fouls & Cards ({HOME_TEAM} Home vs {AWAY_TEAM} Away)</b>',
            labels={'Value': 'Count', 'Date': 'Date'}, hover_data=['Opponent'], template='plotly_white', height=500
        )
        fig.update_layout(hovermode='x unified')
        fig.show()

🚀 GENERATING 7-CHART REPORT FOR: Arsenal vs Liverpool

